In [5]:
import pandas as pd
import numpy as np
import pickle
import re
import os
from os.path import join
from tqdm import tqdm
from datetime import datetime
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from openpyxl import Workbook
import warnings
warnings.simplefilter("ignore")

# 출력 옵션
pd.options.display.float_format = '{:,.0f}'.format

# dtype 정의
pattern = re.compile('키|코드|계좌|차수')

debt_dtype = {'채무자키':str, '타채무자키':str, '담당자키':str}
account_dtype = {'채무자키':str, '계좌키':str, '타채무자키':str}
grt_dtype = {'채무자키':str, '계좌키':str, '타채무자키':str, '보증인키':str}
rehabilitation_dtype = {'채무자키':str, '계좌키':str, '분납키':str, '사건키':str}
credit_dtype = {"채무자키":str, "계좌키":str, '보증인키':str, '신용회복키' : str, "계좌번호":str, "심의차수":str, "변제금수취계좌":str}
# credit_dtype = {'채무자키':str, '계좌키':str, '보증인키':str, '심의차수':str, '변제금\n수취계좌':str}
# credit_dtype = {'채무자키' : str, '계좌키' : str, '보증인키':str, '신용회복키' : str, '심의차수' : str, '조정전이율' : str, '조정후이율' : str, '총상환기간' : str, '유예기간' : str, '원금균등상환기간' : str, '원리균등상환기간' : str, '이자상환기간' : str, '납입회차' : str, '연체기간' : str, '원금균등시작회차' : str, '원금균등종료회차' : str, '원리균등시작회차' : str, '원리균등종료회차' : str, '이자상환시작회차' : str, '이자상환종료회차' : str, '감면율' : str, '주채무자주민번호' : str, '시작회차1' : str, '종료회차1' : str, '채무조정1적용이율1' : str, '채무조정2적용이율1' : str, '시작회차2' : str, '종료회차2' : str, '채무조정1적용이율2' : str, '채무조정2적용이율2' : str, '시작회차3' : str, '종료회차3' : str, '채무조정1적용이율3' : str, '채무조정2적용이율3' : str, '시작회차4' : str, '종료회차4' : str, '채무조정1적용이율4' : str, '채무조정2적용이율4' : str, '시작회차5' : str, '종료회차5' : str, '채무조정1적용이율5' : str, '채무조정2적용이율5' : str, '시작회차6' : str, '종료회차6' : str, '채무조정1적용이율6' : str, '채무조정2적용이율6' : str, '시작회차7' : str, '종료회차7' : str, '채무조정1적용이율7' : str, '채무조정2적용이율7' : str, '시작회차8' : str, '종료회차8' : str, '채무조정1적용이율8' : str, '채무조정2적용이율8' : str, '시작회차9' : str, '종료회차9' : str, '채무조정1적용이율9' : str, '채무조정2적용이율9' : str, '시작회차10' : str, '종료회차10' : str, '채무조정1적용이율10' : str, '채무조정2적용이율10' : str, '재조정횟수' : str, '수정조정횟수' : str, '일시납감면율' : str, '거치기간' : str, '체증전구간의시작회차' : str, '체증전구간의종료회차' : str, '체증전구간의기준기간' : str, '변제금수취계좌' : str, '담보권실행유예기간':str
# }
event_dtype = {'채무자키':str, '법조치키':str, '계좌키':str, '관련법조치키':str, '법취하키':str, '타법조치키':str, '타채무자키':str, '관할법원코드':str}
deposit_dtype = {'채무자키':str, '입금키':str, '계좌키':str, '계좌번호':str, '입금고정키':str, '타채무자키':str}

### LM035(매각채권) - 그냥 수작업
- 매각이 여러건이면 읽을 파일이 많고, 단건이면 수작업도 금방 끝남

### LM036(매입채권 : 관리채권)

In [63]:
# 전체 원데이터 읽기

######################################################
path_raw_data = r"D:\3.자산\전산 dataset\솔림\240329\pkl"
fn_raw_data = "2.솔림_전체_원데이터_240329금감원용"
######################################################

raw_data = pd.read_pickle(join(path_raw_data, fn_raw_data + ".pkl"))

# 종료, 종료요청 제외
raw_data = raw_data.query('~새채무상태.str.contains("종료")')
raw_data.reset_index(inplace=True, drop=True)
print(f'채권수 : {len(raw_data):,} / 반영OPB합 : {raw_data.반영OPB.sum():,}')

# 원채권사, 매각사 코드 merge
# 채무자조회파일에서 매각사 불러오기
file_list = os.listdir(path_raw_data)
fn_debt = [file for file in file_list if file.startswith("채무자조회")][0]

debt = pd.read_pickle(join(path_raw_data, fn_debt))[["채무자키", "매각사"]]

# 코드 파일 불러오기
원채권사코드 = pd.read_excel(r"D:\3.자산\금감원\기관코드표.xlsx", sheet_name="원채권사", dtype={"원채권사코드":str})
매각사코드 = pd.read_excel(r"D:\3.자산\금감원\기관코드표.xlsx", sheet_name="매각사", dtype={"매각사코드":str})

# 병합
raw_data = raw_data.merge(debt, on="채무자키", how="left")
raw_data = raw_data.merge(원채권사코드, on="원채권사", how="left")
raw_data = raw_data.merge(매각사코드, on="매각사", how="left")

# 법조치 읽기
fn_event = [file for file in file_list if file.startswith("법조치조회")][0]
event = pd.read_pickle(join(path_raw_data, fn_event))[["채무자키", "법조치구분", "접수일"]]
len(event)


# 새채무자키 merge
event = pd.merge(event, raw_data.drop_duplicates('채무자키')[["채무자키","새채무자키"]] , on="채무자키", how='left')

# 법조치 필터링(집행권원만 할거면 자산정리_통합파일에서, **신청일 살려야)
# 새채무자키 없는 거 삭제(종결건 삭제)
event.query('~새채무자키.isna()')
# 접수일 없는 거 삭제
event = event.query('접수일 != ""')
# 법조치구분 선택
comp = r"가?압류|가?처분|강제|경매|집행|담보|보전|본안|사해행위|시효연장|유체|재산관계|재산명시|전자독촉|지급명령|불이행|추심"
event = event.query('법조치구분.str.contains(@comp)')

# 접수일 역정렬
event.sort_values("접수일", ascending=False, inplace =True)
event.drop_duplicates(subset='새채무자키', keep='first', inplace=True)

# raw_data에 열추가(merge)
raw_data = raw_data.merge(event[["새채무자키", "접수일"]], on="새채무자키", how="left").fillna("")

# 병합잘못되어 계좌키 중복건 있는지 체크
raw_data[raw_data.계좌키.duplicated()]

채권수 : 32,383 / 반영OPB합 : 403,172,898,431
32383
32383


In [65]:
raw_data.columns

Index(['채무자키', '계좌키', '타채무자키', '채권구분', '채권상태', '새채무상태', '채무상태', '담당자',
       '새보증인상태', '반영OPB', '성명', '주민등록번호', '계좌번호', '원채권사', '매각사구분', '계정과목',
       '은행대출원금', '최초원금', '최초법비용', '최초미수이자', '연체이자율', '현재원금', '현재법비용', 'OPB',
       '현재미수이자', '현재연체이자', '현재합계', '회수합계', '최초대출일', '대출만기일', '최초연체일', '매입일',
       '시효실사결과', '시효완성월', '시효사유', '시효완성일', '종결일', '메모', '원인서류유무', '개인법인', '연령',
       '우편번호', '주소', '새채무자키', '보증인성명인', '보증인상태', '새보증인상태by새채무자키', '접수일_x',
       '최종거래일', '분납연체일', '매각사', '원채권사코드', '매각사코드', '접수일_y'],
      dtype='object')

In [68]:
# 복붙용 파일 만들기
lm036 = pd.DataFrame()

lm036["고객번호"] = raw_data.채무자키
lm036["계약번호"] = raw_data.계좌번호
lm036["차주명"] = raw_data.성명
lm036["차주실명번호"] = raw_data.주민등록번호.apply(lambda x: re.sub("-","", x)+"000" if len(re.sub("-","", x)) == 10 else re.sub("-","", x))
lm036["차주구분"] = np.where(raw_data.개인법인=="법인", "03", np.where(raw_data.주민등록번호.str[7].astype(int) % 2 == 1, "01", "02"))
lm036["매입회차"] = ""
lm036["매입일"] = raw_data.매입일
lm036["대출일"] = raw_data.최초대출일
lm036["대출종류"] = "01"
lm036["상환금액"] = raw_data.회수합계
lm036["만기일"] = raw_data.대출만기일
lm036["최초계약금액"] = raw_data.은행대출원금
lm036["매입서류상잔액"] = raw_data.최초원금 + raw_data.최초법비용
lm036["실제매입금액"] = ""
lm036["기준일장부가액"] = raw_data.현재원금 + raw_data.현재법비용
lm036["금리"] = raw_data.연체이자율
lm036["최종수령일"] = raw_data.최종거래일
lm036["이자연체일수"] = 0
lm036["만기연체일수"] = 0
lm036["시효완성일"] = raw_data.시효완성일
lm036["지급명령신청일"] = raw_data.접수일
lm036["채권서류존재유무"] = np.where(raw_data.원인서류유무==0,"N", "Y")

x = raw_data.새채무상태
y = raw_data.채무상태
z = raw_data.분납연체일
conds = [x.str.contains("개인회생"), x.str.contains("신용회복"), y.str.contains("사망"), ~z.isna()]
values = ["03", "04", "07", "05"]
lm036["현상태"] = np.select(conds, values, default="02")
lm036["원인채권회사번호"] = raw_data.원채권사코드
lm036["원인채권회사명"] = raw_data.원채권사
lm036["채권양도회사번호"] = raw_data.매각사코드
lm036["양도금융회사명"] = raw_data.매각사
lm036["채권추심위임일"] = ""
lm036["채권추심위임기관"] = ""
lm036["연대보증여부"] = "N"

In [69]:
# 엑셀로 보내기
lm036.to_excel(r"D:\3.자산\금감원\lm036.xlsx", index=False)